# Import Statements

In [6]:
import pandas as pd
import json
from collections import Counter
import string
import os

# Program Sketch

**Goal**: To determine the unique set of words that two given people use together (that a given person rarely uses with other people).

**Super High Level**: Find all conversations, scrape JSON for base-case words, scrape rest of JSONs to determine core set of words, pull words on end of frequency chart (1? 2? s.d.)

**High Level Task List**:
1. Pull messenger data for a given person and convert into a DataFrame.
2. Clean up the DataFrame so it can be interacted with easily.
3. Parse words in DataFrame for the user to create initial data set of words.
4. Repeat this process for 10 other people, to start.
5. Determine which words show up in 10/10, 9/10, etc cases.
6. Test this on 10 other people, 100 other people, all cases (if time allows)

# Functions

In [7]:
def clean_dataframe(df):
    # TODO: sometimes a message dataframe doesn't have a 'content' column.
    
    # sort by oldest message
    df.sort('timestamp_ms', inplace=True)
    
    # set all content to lowercase for easier parsing going forward
    df['content'] = df['content'].str.lower()
    df['content'].fillna(value='', inplace=True)
    
    df['date'] = pd.to_datetime(df['timestamp_ms'], unit='ms')
    return df

In [8]:
def convert_filename_to_df(file_name):
    # open the data
    with open(file_name) as f: #
        data = json.load(f)
        
    # pull out messages for dataframe conversion
    messages = [x for x in data['messages']]
    
    df = pd.DataFrame(messages)
    return df

In [9]:
def person(df, name):
    # this assumes you've already created chat_df
    unique_names = df['sender_name'].unique()
    if name in unique_names:
        return df[df['sender_name'] == name]
    else:
        #print name, 'isn\'t in the DataFrame. \nTry one of these:', ', '.join(unique_names.astype(str))
        return pd.DataFrame(columns=test_df.columns)

In [10]:
def check_this(df, max_limit=100000000, min_limit=0):
    len_num = len(person(df, my_name))
    # check all of these things in order of 'ew, bye'ness:
    # * if Ben is in the conversation
    # * if I've said very little in the conversation (less than 100?)
    if (('Ben Knight' in set(df['sender_name']))
        or (len_num >= max_limit)
        or (len_num < min_limit)
        ):
        return False
    return True

In [11]:
def generate_wordlist(file_name, name, stop_file=False, way='common'):
    # Let's parse our data.
    chat_df_raw = convert_filename_to_df(file_name)
    chat_df = clean_dataframe(chat_df_raw)

    # Pull out only my chats
    my_chats = person(chat_df, name)

    # pick name_word. if it's you, use 'you'. else, use their first name
    name_word = name.lower().split()[0] if name != my_name else 'you' 
    sent_str = '{} sent a'.format(name_word)
    
    # pull only my sentences out
    convos = my_chats[my_chats['content'].apply(lambda x: sent_str not in x)]['content']
    
    # unicode word list
    unicode_list = " ".join(convos.str.lower()).split()
    
    # convert each word to a string and remove punctuation.
    sample_words = [x.encode('UTF8').translate(None, string.punctuation) for x in unicode_list if ('http' not in x) and (not x.startswith('www'))]
    
    # counter set of those words, removing empty spaces
    base_words = Counter([w for w in sample_words if w != ''])
    
    if way == 'common':
        return base_words
    elif way == 'count':
        return [(k, v) for k, v in base_words.iteritems()]
    elif way == 'percent':
        return [(k, round(1000.0*v/sum(base_words.values()),5)) for k, v in base_words.iteritems()]
            

In [4]:
file_name = main_file
name = my_name
stop_file=True
way='common'

# Let's parse our data.
chat_df_raw = convert_filename_to_df(file_name)
chat_df = clean_dataframe(chat_df_raw)

# Pull out only my chats
my_chats = person(chat_df, name)

# pick name_word. if it's you, use 'you'. else, use their first name
name_word = name.lower().split()[0] if name != my_name else 'you' 
sent_str = '{} sent a'.format(name_word)

# pull only my sentences out
convos = my_chats[my_chats['content'].apply(lambda x: sent_str not in x)]['content']

# unicode word list
unicode_list = " ".join(convos.str.lower()).split()

# convert each word to a string and remove punctuation.
sample_words = [x.encode('UTF8').translate(None, string.punctuation) for x in unicode_list if ('http' not in x) and (not x.startswith('www'))]

# counter set of those words, removing empty spaces
base_words = Counter([w for w in sample_words if w != ''])

NameError: name 'main_file' is not defined

In [234]:
sample_words2 = [x.encode('UTF8').translate(None, string.punctuation) for x in unicode_list if (x not in stops['word']) and ('http' not in x) and (not x.startswith('www'))]

In [264]:
base_words3 = Counter([w for w in sample_words if (w != '') and (w not in stops2['word'].values)])

In [265]:
len(base_words3)

12523

In [267]:
len(stops2)

1546

In [266]:
base_words3.most_common(30)

[('adore', 71),
 ('hows', 63),
 ('print', 40),
 ('gin', 38),
 ('smile', 36),
 ('\xc3\xb0\xc2\x9f\xc2\x98\xc2\x8d', 33),
 ('slept', 33),
 ('wake', 32),
 ('butt', 31),
 ('signing', 29),
 ('movies', 28),
 ('union', 27),
 ('charge', 27),
 ('chrissy', 26),
 ('commute', 26),
 ('episodes', 25),
 ('shows', 25),
 ('alarm', 25),
 ('impressed', 25),
 ('stephs', 24),
 ('workout', 24),
 ('caught', 24),
 ('manager', 24),
 ('cleaning', 23),
 ('enjoy', 23),
 ('classes', 23),
 ('kick', 23),
 ('arrived', 23),
 ('closet', 23),
 ('warm', 22)]

In [248]:
len(base_words), len(base_words2), sample_words[0]

(14062, 13859, 'oh')

In [237]:
len(sample_words), len(sample_words2)

(231016, 231016)

In [15]:
stops2 = pd.read_csv('stop_file2.csv')

In [17]:
generate_wordlist(main_file, my_name, stops2).most_common(100)

[('i', 10053),
 ('and', 6068),
 ('you', 6029),
 ('to', 5493),
 ('the', 5427),
 ('a', 4395),
 ('3', 3743),
 ('it', 3309),
 ('my', 2952),
 ('so', 2683),
 ('im', 2421),
 ('of', 2374),
 ('that', 2360),
 ('for', 2355),
 ('in', 2136),
 ('but', 1985),
 ('is', 1887),
 ('like', 1756),
 ('was', 1610),
 ('me', 1592),
 ('on', 1563),
 ('this', 1525),
 ('have', 1479),
 ('its', 1471),
 ('just', 1460),
 ('with', 1264),
 ('do', 1222),
 ('be', 1148),
 ('can', 1114),
 ('at', 1096),
 ('if', 1076),
 ('are', 979),
 ('your', 960),
 ('not', 949),
 ('we', 946),
 ('up', 928),
 ('good', 908),
 ('about', 892),
 ('know', 845),
 ('what', 789),
 ('one', 777),
 ('out', 775),
 ('love', 774),
 ('all', 751),
 ('also', 718),
 ('time', 677),
 ('get', 664),
 ('or', 664),
 ('now', 648),
 ('how', 642),
 ('then', 620),
 ('when', 617),
 ('as', 592),
 ('did', 589),
 ('dont', 583),
 ('really', 569),
 ('thats', 561),
 ('oh', 554),
 ('from', 548),
 ('going', 539),
 ('think', 537),
 ('omg', 526),
 ('some', 515),
 ('lol', 514),
 ('a

# User Inputs

In [12]:
my_name = 'Cassie Beth'
main_name = 'Ben Knight'
main_file = 'BenKnight/message.json'
inbox_url = "/Users/clebauer/Downloads/facebook-cassielebauer/messages/inbox"

# Parsing the Messenger Data

In [13]:
# pull main dataset
main_df = convert_filename_to_df(main_file)
main_data = generate_wordlist(main_file, main_name)

In [14]:
# for each folder of conversations, pull the data.
# then, perform some checks to see if we want to use this dataframe or not.
#for num_limit in range(0,1001,50):
min_limit = 500
#max_limit = 500
count = {'good':0,'bad':0}

for human in os.listdir(inbox_url)[1:]:
    file_name = '/'.join([inbox_url, human, 'message.json'])
    #print file_name
    test_df = convert_filename_to_df(file_name)
    if check_this(test_df, min_limit=min_limit):
        count['good'] += 1
        #print '{}, len: {}'.format(human, len(test_df))
    else:
        count['bad'] += 1
        #print 'Bad: {}'.format(human)

#    print 'With message limit {}, {}'.format(num_limit, count)

OSError: [Errno 2] No such file or directory: '/Users/clebauer/Downloads/facebook-cassielebauer/messages/inbox'

In [18]:
generate_wordlist(file_name, my_name).most_common(5)

NameError: name 'file_name' is not defined

In [124]:
type(generate_wordlist(file_name, my_name)), type(generate_wordlist(file_name, my_name).most_common(5))

(collections.Counter, list)

In [193]:
# for each folder of conversations, pull the data.
# then, perform some checks to see if we want to use this dataframe or not.
#for num_limit in range(0,1001,50):
min_limit = 500
#max_limit = 500
count = {'good':0,'bad':0}
full_tbl = pd.DataFrame(columns=['word'])
#mc_count = 100 .most_common(mc_count)

for human in os.listdir(inbox_url)[1:]:
    file_name = '/'.join([inbox_url, human, 'message.json'])
    #print file_name
    test_df = convert_filename_to_df(file_name)
    if check_this(test_df, min_limit=min_limit):
        test_words = [(k, v) for k, v in generate_wordlist(file_name, my_name, way='percent')]
        test_tbl = pd.DataFrame(test_words, columns=['word', human.split('_')[0]])
        full_tbl = pd.merge(full_tbl, test_tbl, on = 'word', how='outer')
        #print '{}, len: {}'.format(human, len(test_df))

print 'With message limit {}, {}'.format(num_limit, count)

With message limit 500, {'bad': 0, 'good': 0}


In [194]:
full_tbl = full_tbl.set_index(full_tbl['word']).drop(['word'], axis=1).fillna(0)

In [198]:
full_tbl['total'] = full_tbl.mean(axis=1)

In [199]:
full_tbl = full_tbl.sort('total', ascending=False)

In [156]:
18.505418 + 208.076851# + 208.076851

226.582269

In [214]:
pd.DataFrame(stop_tbl.reset_index()['word']).to_csv('stop_file.csv', index=False, header=True)

In [260]:
stop_tbl = full_tbl[full_tbl['total'] > full_tbl['total'].mean()]# + full_tbl['total'].std()]

In [1]:
len(stop_tbl)

NameError: name 'stop_tbl' is not defined

In [262]:
pd.DataFrame(stop_tbl.reset_index()['word']).to_csv('stop_file2.csv', index=False, header=True)

In [54]:
test_line = '/Users/clebauer/Downloads/facebook-cassielebauer/messages/inbox/eowynwinchester_txpo2btkkg/message.json'


In [121]:
test_df.sort('timestamp_ms')

,content,photos,sender_name,share,sticker,timestamp_ms,type
55,I was told to contact you with the scam details,NaN,Zachary Knight,NaN,NaN,1517855610639,Generic
54,?,NaN,Zachary Knight,NaN,NaN,1517855640084,Generic
53,You can now call each other and see informatio...,NaN,Cassie Beth,NaN,NaN,1517857088827,Generic
52,"Heya! Yea, that would be great! Indeed takes s...",NaN,Cassie Beth,NaN,NaN,1517857136971,Generic
51,"Awesome, should i wait for the info or tell yo...",NaN,Zachary Knight,NaN,NaN,1517857297242,Generic
50,Feel free to sent it along my way while I scra...,NaN,Cassie Beth,NaN,NaN,1517857316981,Generic
49,"Will do. The company was Lennar Corporation, t...",NaN,Zachary Knight,NaN,NaN,1517857572476,Generic
48,I have filed an fbi report an contacted Lennar...,NaN,Zachary Knight,NaN,NaN,1517857602576,Generic
47,Also i had applied to a lennar listing on inde...,NaN,Zachary Knight,NaN,NaN,1517857883510,Generic
46,Would you happen to have the link to that job? :),NaN,Cassie Beth,NaN,NaN,1517858309185,Generic


In [ ]:
# Fun Facts of Learning:
#
# # number of "words" I typed to Ben for the first year we talked.
# len([x.translate(None, string.punctuation) for x in sample_words])
# > 231,132
#
# # number of unique words with punctuation
# len(set(sample_words))
# > 24,259
#
# # number of unique words, no punctuation!
# len(set([x.translate(None, string.punctuation) for x in sample_words]))
# > 14,176

# Fun Facts:
# strip out punctuation
# strip out http links

# think about:
# \xc3\xa3\xc2\xa1
# 3 instead of <3
# misspellings
# long one sentence, many words
